## Initial exploratory analysis and generation of a baseline model

### 1) Data cleaning and pre-processing

In [ ]:
# import packages
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import math
import gc

In [ ]:
# load data (train.csv will be the complete dataset for predictive modeling, ignore Test.csv for now)
data=pd.read_csv("./data/Train.csv")

In [ ]:
# print the first five observations
data.head()

In [ ]:
data.info()

A quick examination shows that the dataset contains an ID column, a location column indicating the sensor location, 6 feature variables containing sensor data to be used for target prediction, as well as the target itself.

The 6 feature variables are: temperature, precipitation, relative humidity, wind direction, wind speed, atmospheric pressure

For each observation, the data for the feature variables reflects the raw sensor measurements collected hourly over a 5-day period. These measurement periods are stored as a string per observation / feature. They thus need to be unpacked into list. As can be seen from the data table, each observation period contains varying degrees of missings (NaN), which need to be managed in a proper way. These missings occur either in the form of 'nan' or spaces in the string.
Also, it would be good to extract summary statistics from the different features for each recording period (e.g. mean, standard deviation, minimum, maximum) in order to engineer new features for the prediction.

The target represents the amount of PM2.5 particles in ug/m^3 measured exactly 24h after the end of the recording period for the feature measurements.

#### Convert the sensor data for each feature from string type into a list of values

In [ ]:
# define a function to replace spaces in the string (i.e. missings) with NaN
def replace_nan(x):
    if x==" ":
        return np.nan
    else :
        return float(x)

# define list of feature names
features=["temp","precip","rel_humidity","wind_dir","wind_spd","atmos_press"]


for feature in features : 
    # first replace every 'nan' in a cell with an empty space, split using comma, and then apply replace_nan function on every item
    data[feature]=data[feature].apply(lambda x: [ replace_nan(X) for X in x.replace("nan"," ").split(",")])    

In [ ]:
data.head()

Thus, for every observation, each feature value is represented as a list of raw values over an hourly 5-day recording period

### Features engineering part

#### Extract summary statistics per observation period for every feature

In [ ]:
df_temp = data[['location', 'temp']]
df_temp.head()

In [ ]:
df_temp['percent_nan'] = data.temp.apply(lambda x: np.isnan(np.array(x)).sum()/len(x)*100)

In [ ]:
df_temp.head()

In [ ]:
# recording periods contain varying degrees of NaNs: compute percent NaN for each recording period and feature
# function to compute the percentage of NaNs per recording period
def compute_percent_nan(df, col_name):
    df['percent_nan_'+col_name] = df[col_name].apply(lambda x: np.isnan(np.array(x)).sum()/len(x)*100)
    return df

In [ ]:
# calculate percentage of missings per recording period and feature and append to dataframe
for col_name in tqdm(features):
    data=compute_percent_nan(data,col_name)

In [ ]:
data.head()

In [ ]:
data.columns.tolist()

In [ ]:
# aggregation function extracting summary statistics from every recording period and appending it as a new column to a dataframe
def aggregate_features(x,col_name):
    x["max_"+col_name]=x[col_name].apply(np.max)
    x["min_"+col_name]=x[col_name].apply(np.min)
    x["mean_"+col_name]=x[col_name].apply(np.mean)
    x["std_"+col_name]=x[col_name].apply(np.std)
    x["var_"+col_name]=x[col_name].apply(np.var)
    x["median_"+col_name]=x[col_name].apply(np.median)
    x["ptp_"+col_name]=x[col_name].apply(np.ptp)
    return x  

# function returning only non-Null values (helper for aggregation function)
def remove_nan_values(x):
    return [e for e in x if not math.isnan(e)]


In [ ]:
# remove NaNs from dataframe
for col_name in tqdm(features):
   data[col_name]=data[col_name].apply(remove_nan_values)

In [ ]:
data.head()

In [ ]:
#extract summary statistics for each recording period and feature
for col_name in tqdm(features):
    data=aggregate_features(data,col_name)

In [ ]:
data.head()

In [ ]:
# extract all single raw values from each feature and append them as new columns to the dataframe
for x in range(121):
    data["newtemp"+ str(x)] = data.temp.str[x]
    data["newprecip"+ str(x)] = data.precip.str[x]
    data["newrel_humidity"+ str(x)] = data.rel_humidity.str[x]
    data["newwind_dir"+ str(x)] = data.wind_dir.str[x]
    data["windspeed"+ str(x)] = data.wind_spd.str[x]
    data["atmospherepressure"+ str(x)] = data.atmos_press.str[x]

In [ ]:
# drop raw sensor data contained as list from the initial dataset
data.drop(features,1,inplace=True)

In [ ]:
data.head()

### Exploratory data analysis

In [ ]:
# list dataframe columns for target and condensed features, including percent NaNs
summary_columns = data.columns[:51]
summary_columns

In [ ]:
# select data with summary statistics for the different features
df_condensed = data[summary_columns]
df_condensed.set_index('ID', drop=True, inplace=True)
df_condensed.head()

### Compute basic summary statistics

In [ ]:
df_condensed.info()

In [ ]:
# get descriptive statistics on the percentage of NaNs for the recording period
df_condensed[['percent_nan_temp', 'percent_nan_precip',
       'percent_nan_rel_humidity', 'percent_nan_wind_dir',
       'percent_nan_wind_spd', 'percent_nan_atmos_press']].describe().round(2)

### Distribution and correlations of percent NaNs between features

In [ ]:
# check the overall distribution of percent NaNs depending on sensor location using histograms

nan_columns = ['percent_nan_temp', 'percent_nan_precip','percent_nan_rel_humidity', 'percent_nan_wind_dir','percent_nan_wind_spd', 
        'percent_nan_atmos_press']

fig = plt.figure(figsize=(15, 20))

for i in range(1, len(nan_columns)+1): # start with i=1 (0th subplot is not possible)
    ax = fig.add_subplot(4, 2, i) # arrange figure as rows = 6 x cols = 4 panel and add ith subplot
    subplot = sns.histplot(x=nan_columns[i-1], hue='location', bins=20, data=df_condensed)
    ax.set_xlabel(nan_columns[i-1])

fig.tight_layout() # prevents subplots from overlapping

In [ ]:
# plot pairplot to examine feature-wise correlations in the percent NaNs
sns.pairplot(df_condensed[['location', 'percent_nan_temp', 'percent_nan_precip','percent_nan_rel_humidity', 'percent_nan_wind_dir','percent_nan_wind_spd', 
        'percent_nan_atmos_press']], hue='location');

### Filter dataframe for observations with percent NaN < 30% for all features

In [ ]:
# filter observations based on percent NaN and check again the data distribution of the target and summary features
df_filtered = df_condensed[(df_condensed[nan_columns]<30).all(axis=1)]
df_filtered.head()

In [ ]:
percent_reduc = (df_condensed.shape[0]-df_filtered.shape[0]) / df_condensed.shape[0]*100
print(f'Percent reduction in dataset size after filtering: {round(percent_reduc,1)}%')

### Data distribution for the summary features and target

In [ ]:
# get target and summary feature columns
colnames = df_condensed.columns.to_list()[1::] # get columns w/o location column
del colnames[1:8] # delete columns containing data on percent NaN
colnames

In [ ]:
# plot histograms on the distribution of the other data
fig = plt.figure(figsize=(15, 20))

for i in range(1, len(colnames)+1): # start with i=1 (0th subplot is not possible)
    ax = fig.add_subplot(9, 5, i) # arrange figure as rows = 6 x cols = 4 panel and add ith subplot
    subplot = sns.histplot(x=colnames[i-1], hue='location', data=df_condensed)
    ax.set_xlabel(colnames[i-1])

fig.tight_layout() # prevents subplots from overlapping

### Mean differences in air pollution between locations for filtered and unfiltered data

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(9, 5))
plt.suptitle('Air pollution depending on sensor location')

plot1 = sns.boxplot(data=df_condensed, x='location', y='target', ax=ax1, order=['A', 'B', 'C', 'D', 'E'])
ax1.set_title('unfiltered')
ax1.set_ylabel('PM2.5 (ug/m^3')
ax1.set_xlabel('Sensor location')

plot2 = sns.boxplot(data=df_filtered, x='location', y='target', ax=ax2, order=['A', 'B', 'C', 'D', 'E'])
ax2.set_title('filtered')
ax2.set_ylabel('PM2.5 (ug/m^3)')
ax2.set_xlabel('Sensor location')

plt.tight_layout();

### Correlations between mean values for features and target for unfiltered and filtered data

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(9, 5))
plt.suptitle('Relationship between mean temperature and PM2.5')

plot1 = sns.scatterplot(data=df_condensed, x='mean_temp', y='target', hue='location', ax=ax1, hue_order=['A', 'B', 'C', 'D', 'E'])
plot1.set_title('unfiltered')

plot2 = sns.scatterplot(data=df_filtered, x='mean_temp', y='target', hue='location', ax=ax2, hue_order=['A', 'B', 'C', 'D', 'E'])
plot2.set_title('filtered')

plt.tight_layout()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(9, 5))
plt.suptitle('Relationship between mean precipitation and PM2.5')

plot1 = sns.scatterplot(data=df_condensed, x='mean_precip', y='target', hue='location', ax=ax1, hue_order=['A', 'B', 'C', 'D', 'E'])
plot1.set_title('unfiltered')

plot2 = sns.scatterplot(data=df_filtered, x='mean_precip', y='target', hue='location', ax=ax2, hue_order=['A', 'B', 'C', 'D', 'E'])
plot2.set_title('filtered')

plt.tight_layout()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(9, 5))
plt.suptitle('Relationship between mean humidity and PM2.5')

plot1 = sns.scatterplot(data=df_condensed, x='mean_rel_humidity', y='target', hue='location', ax=ax1, hue_order=['A', 'B', 'C', 'D', 'E'])
plot1.set_title('unfiltered')

plot2 = sns.scatterplot(data=df_filtered, x='mean_rel_humidity', y='target', hue='location', ax=ax2, hue_order=['A', 'B', 'C', 'D', 'E'])
plot2.set_title('filtered')

plt.tight_layout()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(9, 5))
plt.suptitle('Relationship between mean wind direction and PM2.5')

plot1 = sns.scatterplot(data=df_condensed, x='mean_wind_dir', y='target', hue='location', ax=ax1, hue_order=['A', 'B', 'C', 'D', 'E'])
plot1.set_title('unfiltered')

plot2 = sns.scatterplot(data=df_filtered, x='mean_wind_dir', y='target', hue='location', ax=ax2, hue_order=['A', 'B', 'C', 'D', 'E'])
plot2.set_title('filtered')

plt.tight_layout()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(9, 5))
plt.suptitle('Relationship between mean wind speed and PM2.5')

plot1 = sns.scatterplot(data=df_condensed, x='mean_wind_spd', y='target', hue='location', ax=ax1, hue_order=['A', 'B', 'C', 'D', 'E'])
plot1.set_title('unfiltered')

plot2 = sns.scatterplot(data=df_filtered, x='mean_wind_spd', y='target', hue='location', ax=ax2, hue_order=['A', 'B', 'C', 'D', 'E'])
plot2.set_title('filtered')

plt.tight_layout()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(9, 5))
plt.suptitle('Relationship between mean atmospheric pressure and PM2.5')

plot1 = sns.scatterplot(data=df_condensed, x='mean_atmos_press', y='target', hue='location', ax=ax1, hue_order=['A', 'B', 'C', 'D', 'E'])
plot1.set_title('unfiltered')

plot2 = sns.scatterplot(data=df_filtered, x='mean_atmos_press', y='target', hue='location', ax=ax2, hue_order=['A', 'B', 'C', 'D', 'E'])
plot2.set_title('filtered')

plt.tight_layout()

### Calculate the basemodel
We compute a baseline model, predicting the target using the mean values of the different features.

In [ ]:
# import libraries and metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error

In [ ]:
# get target and features
X = df_condensed[['location', 'mean_temp', 'mean_precip', 'mean_rel_humidity', 'mean_wind_dir', 'mean_wind_spd', 'mean_atmos_press']]
y = df_condensed.target

print(X.shape)
print(y.shape)

In [ ]:
# perform train test split, stratified by location in order to ensure that locations are balanced between training and test set
rseed = 42

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=X['location'], random_state=rseed)

print(X_train.shape)
print(X_test.shape)

In [ ]:
# scale features using z-transformation
scaler = StandardScaler()

# fit_transform training data, drop location column since it will not be used for prediction
X_train_scaled = scaler.fit_transform(X_train.drop('location', axis=1))
# aply transform to test data
X_test_scaled = scaler.transform(X_test.drop('location', axis=1))

In [ ]:
print(X_train_scaled.shape)
print(X_test_scaled.shape)

In [ ]:
# check if transformation worked

# calculate mean
print('Means')
for i in range(X_train_scaled.shape[1]):
    print(X_train_scaled[:, i].mean().round(2))

print('\nStandard deviations:')
# calculate standard deviation
for i in range(X_train_scaled.shape[1]):
    print(X_train_scaled[:, i].std().round(2))

In [ ]:
# train the model
linreg = LinearRegression()

linreg.fit(X_train_scaled, y_train)

In [ ]:
# get beta coefficients
linreg.coef_

In [ ]:
# predict new cases
y_pred = linreg.predict(X_test_scaled)

# show first 10 predictions
y_pred[:10]

In [ ]:
# evaluate model accurray

rmse_linreg = mean_squared_error(y_test, y_pred, squared=False)
r2_linreg = r2_score(y_test, y_pred)

print(f'RMSE on testset: {round(rmse_linreg,2)}')
print(f'Coefficient of determination on testset: {round(r2_linreg,2)}')

### Calculate baseline model only for location B 
We re-perform our linear regression only using sensor data for location B, because the data for this location seem to be most representative of the whole data, without risking our model training being confounded by location-specific data due to the hierarchical data structure.

In [ ]:
# extract data only for location B
df_locB = df_condensed[df_condensed.location == 'B']
df_locB.head()

In [ ]:
df_locB.shape

In [ ]:
# get target and features
X_locB = df_locB[['location', 'mean_temp', 'mean_precip', 'mean_rel_humidity', 'mean_wind_dir', 'mean_wind_spd', 'mean_atmos_press']]
y_locB = df_locB.target

In [ ]:
# perform train test split, stratification not needed anymore
rseed = 42

X_train_locB, X_test_locB, y_train_locB, y_test_locB = train_test_split(X_locB, y_locB, test_size=0.3, random_state=rseed)

print(X_train_locB.shape)
print(X_test_locB.shape)

In [ ]:
# scale features using z-transformation
scaler_locB = StandardScaler()

# fit_transform training data, drop location column since it will not be used for prediction
X_train_scaled_locB = scaler.fit_transform(X_train_locB.drop('location', axis=1))
# aply transform to test data
X_test_scaled_locB = scaler.transform(X_test_locB.drop('location', axis=1))

In [ ]:
# train the model
linreg_locB = LinearRegression()

linreg_locB.fit(X_train_scaled_locB, y_train_locB)

In [ ]:
# get beta coefficients
linreg_locB.coef_

In [ ]:
# predict new cases
y_pred_locB = linreg.predict(X_test_scaled_locB)

# show first 10 predictions
y_pred_locB[:10]

In [ ]:
# evaluate model accurray

rmse_linreg_locB = mean_squared_error(y_test_locB, y_pred_locB, squared=False)
r2_linreg_locB = r2_score(y_test_locB, y_pred_locB)

print(f'RMSE on testset: {round(rmse_linreg_locB,2)}')
print(f'Coefficient of determination on testset: {round(r2_linreg_locB,2)}')

### Calculate baseline model with filtered data

In [ ]:
# get target and features
X_filtered = df_filtered[['location', 'mean_temp', 'mean_precip', 'mean_rel_humidity', 'mean_wind_dir', 'mean_wind_spd', 'mean_atmos_press']]
y_filtered = df_filtered.target

In [ ]:
# perform train test split, stratify according to sensor location
rseed = 42

X_train_f, X_test_f, y_train_f, y_test_f = train_test_split(X_filtered, y_filtered, test_size=0.3, 
                                            stratify=X_filtered['location'], random_state=rseed)

print(X_train_f.shape)
print(X_test_f.shape)

In [ ]:
# scale features using z-transformation
scaler_f = StandardScaler()

# fit_transform training data, drop location column since it will not be used for prediction
X_train_scaled_f = scaler.fit_transform(X_train_f.drop('location', axis=1))
# aply transform to test data
X_test_scaled_f = scaler.transform(X_test_f.drop('location', axis=1))

In [ ]:
# train the model
linreg_f = LinearRegression()

linreg_f.fit(X_train_scaled_f, y_train_f)

In [ ]:
# get beta coefficients
linreg_f.coef_

In [ ]:
# predict new cases
y_pred_f = linreg.predict(X_test_scaled_f)

# show first 10 predictions
y_pred_f[:10]

In [ ]:
# evaluate model accurray

rmse_linreg_f = mean_squared_error(y_test_f, y_pred_f, squared=False)
r2_linreg_f = r2_score(y_test_f, y_pred_f)

print(f'RMSE on testset: {round(rmse_linreg_f,2)}')
print(f'Coefficient of determination on testset: {round(r2_linreg_f,2)}')

### Residual plot for baseline model trained on filtered data

In [ ]:
# calculate residuals
residual = y_test_f - y_pred_f

# compute mean of residuals
np.mean(residual)

Thus, on average, our baseline model rather seems to underestimate the air pollution, as indicated by an average residual error > 0.

In [ ]:
sns.scatterplot(x=y_pred_f, y=residual, hue=X_test_f['location'])
plt.xlabel('y_pred')
plt.ylabel('residual')
plt.title('Residual plot');

In [ ]:
sns.scatterplot(x=X_test_f.mean_temp, y=residual, hue=X_test_f['location'])
plt.xlabel('mean temperature')
plt.ylabel('residual')
plt.title('Residual plot');

In [ ]:
sns.scatterplot(x=X_test_f.mean_precip, y=residual, hue=X_test_f['location'])
plt.xlabel('mean precipitation')
plt.ylabel('residual')
plt.title('Residual plot');

In [ ]:
sns.scatterplot(x=X_test_f.mean_rel_humidity, y=residual, hue=X_test_f['location'])
plt.xlabel('mean relative humidity')
plt.ylabel('residual')
plt.title('Residual plot');

In [ ]:
sns.scatterplot(x=X_test_f.mean_wind_dir, y=residual, hue=X_test_f['location'])
plt.xlabel('mean wind direction')
plt.ylabel('residual')
plt.title('Residual plot');

In [ ]:
sns.scatterplot(x=X_test_f.mean_wind_spd, y=residual, hue=X_test_f['location'])
plt.xlabel('mean wind speed')
plt.ylabel('residual')
plt.title('Residual plot');

In [ ]:
sns.scatterplot(x=X_test_f.mean_atmos_press, y=residual, hue=X_test_f['location'])
plt.xlabel('mean atmospheric pressure')
plt.ylabel('residual')
plt.title('Residual plot');

### Conclusion
The data has a different hierarchical, i.e. nested structure, with measurements being nested within defined sensor locations. This makes the dataset difficult to handle.


Each observation consists of an 5-day recording period, where sensor data is recorded in an hourly fashion. We extracted summary statistics from those recording periods for each feature, using those statistics as new features to predict air pollution 24 h later, measured as the amount of PM2.5 particles in ug/m^3.


A baseline model using a simple linear regression model using the mean values in the features per recording period to predict air pollution yielded almost no predictive power, as reflected in a **R2==0.05**. Calculating this baseline model only for a single sensor location (location B) also produced no usable prediction power. Notably, training the baseline model on filtered data resulted in an **R2==0.09**. However, it should be stressed that different train-test splits were performed to get training and test sets for the unfiltered and filtered data.

However, the recording periods showing varying degrees of NaNs. An exploratory analysis on the distribution and correlation patterns of the percent NaNs shows that there are some recording periods with vary high percent NaNs (>30%). The percent NaN is usually highly correlated between the different features, thus affecting the whole data collection within a recording period and therefore probably reflecting general sensor malfunctioning. However, there are also some features showing more specific occurrences of NaNs that seem to be additionally associated with the sensor location.

It would be important to define a threshold for the percent NaN to consider an observation reliable enough to extract summary statistics. This threshold can then be used as an exclusion criterion to filter the observations.

**Possible steps to improve prediction accuracy**
1) Log-transform the target variable (seems to show a skewed distribution)
2) Standardize features within each sensor location to avoid cofounds by variables specific to sensor locations
3) Perform hierarchical linear modeling
4) Test different sets of features and/or different types of ML algorithms